观察这100条数据的答案

In [9]:
import json
import pandas as pd
data_path = './benchmark_wiki_counterfact_test_cf.json'
# data_path = './benchmark_ZsRE_ZsRE-test-all.json'
with open(data_path, 'r') as f:
    data_list = json.load(f)

model_name = 'Meta-Llama-3-8B-Instruct'
# model_name = 'gpt-j-6b'
with open(f'answer-counterfact-{model_name}-<=n+1-words.json', 'r') as f:
    answer_list = f.readlines()

answer_compare_dict = {
    'prompt': {},
    'ground_truth': {},
    'answer1': {},
    'answer2': {},
    'answer3': {},
}
for i in range(100):
    answer_compare_dict['prompt'][i] = data_list[i]['prompt']
    answer_compare_dict['ground_truth'][i]= data_list[i]['ground_truth']
    answer_dict = json.loads(answer_list[i])
    answer_compare_dict['answer1'][i] = answer_dict['answer1']
    answer_compare_dict['answer2'][i] = answer_dict['answer2']
    answer_compare_dict['answer3'][i] = answer_dict['answer3']

In [11]:
answer_pd = pd.DataFrame(answer_compare_dict)
# answer_pd[:20]
# answer_pd[20:40]
answer_pd[40:60]

,prompt,ground_truth,answer1,answer2,answer3
40,The place of birth of Charli D'Amelio is,Norwalk,Norwalk,Hershey,United States
41,The name of the award H.E.R. won is,Asia's Most Influential Philippines,the Grammy Award for Best,Grammy Award for Best R,MTV Video Music Award.
42,The name of the country which Hurricane Ida is...,Jamaica,the United,United States,Haiti\n
43,The gender of Abdul Rashid Dostum is,male,male.\n,Male\n\n,Male\n
44,The official language of Ukraine is,Ukrainian,"Ukrainian,",Ukrainian\n\n,Ukrainian\n
45,The name of the child of Isildur is,Elendur,not mentioned,Elend,Elend
46,The gender of Luke Kuechly is,male,male.\n,Male\n,Male\n
47,The name of the award Lionel Messi won is,Best International Athlete ESPY Award,the Ballon d'Or,Ballon d'Or\n,Ballon d'Or\n
48,The name of the sports team which Pravin Tambe...,Sunrisers Hyderabad,Mumbai Indians.,Mumbai Indians\n,Mumbai Indians\n
49,The name of the country of citizenship of LeBr...,United States of America,the United States. He,United States of America.,United States of America.


生成100条数据的答案观察

In [ ]:
prefix_1 = """Q:What is the native language of Christiane Cohendy?\nA:French\n"""
prefix_3 = """Q:What is the native language of Christiane Cohendy?\nA:French\nQ:What is the final year of Atlanta Flames?\nA:1980\nQ:What is Barbara Legrand's position on the field while playing football?\nA:midfielder\n"""

In [ ]:
import transformers
import torch
import json
import os
os.environ['CUDA_VISIBLE_DEVICES']= '0'
# model_name = 'Meta-Llama-3-8B-Instruct'
model_name = 'gpt-j-6b'
model_id = f"/share/huggingface/{model_name}"

In [ ]:
pipeline = transformers.pipeline("text-generation", 
                                 model=model_id, 
                                 model_kwargs={"torch_dtype": torch.bfloat16}, 
                                 device_map="auto",)

In [ ]:
# data_path = './benchmark_wiki_counterfact_test_cf.json'
data_path = './benchmark_ZsRE_ZsRE-test-all.json'
target_new_lens,ground_truth_lens = [],[]
with open(data_path, 'r') as f:
    data_list = json.load(f)

In [ ]:
for data in data_list:
    target_new_lens.append(len(data['target_new'].split(' ')))
    # ground_truth_lens.append(len(data['ground_truth'].split(' ')))
    ground_truth_lens.append(len(data['ground_truth'][0].split(' ')))

sum(target_new_lens)/len(target_new_lens),\
sum(ground_truth_lens)/len(ground_truth_lens),\
max(target_new_lens),max(ground_truth_lens)

In [ ]:
# # TODO:考虑使用检索增强技术,匹配到语义和句式更相近的句子
with open(f'answer-zsre-{model_name}.json', 'a') as f:
    for data in data_list[:100]:
        prompt1 = data['prompt']
        prompt2 = f"{prefix_1}Q:{data['prompt']}\nA:"
        prompt3 = f"{prefix_3}Q:{data['prompt']}\nA:"
        answer1 = pipeline(prompt1, max_new_tokens=10, do_sample=False)
        answer2 = pipeline(prompt2, max_new_tokens=10, do_sample=False)
        answer3 = pipeline(prompt3, max_new_tokens=10, do_sample=False)
        data_dict = {
            'prompt': data['prompt'],
            'answer1': answer1[0]['generated_text'].replace(prompt1, ''),
            'answer2': answer2[0]['generated_text'].replace(prompt2, ''),
            'answer3': answer3[0]['generated_text'].replace(prompt3, ''),
        }

        f.write(json.dumps(data_dict, ensure_ascii=False) + '\n')
        f.flush()